Read CSV file into data frame

In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
events.show(10)

Top 5 products

In [0]:
revenue = events.filter(events.event_type == "purchase").groupBy("brand", "product_id").agg({"price": "sum"}).withColumnRenamed("sum(price)", "revenue")
revenue.orderBy("revenue", ascending=False).limit(5).show()

Running Total per User

In [0]:
from pyspark.sql.functions import sum, col
from pyspark.sql.window import Window
window_spec = Window.partitionBy("user_id").orderBy("event_time")
purchase_events = events.filter(col("event_type")== "purchase")
purchase_events = purchase_events.withColumn("cumulative_total", sum("price").over(window_spec))
purchase_events.orderBy("user_id", "event_time").limit(50).display()

 Conversion rate by category

In [0]:
events.groupBy("category_code").pivot("event_type").count().withColumn("conversion_rate", col("purchase")/col("view") * 100).orderBy("conversion_rate", ascending=False).display()

